In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional
import matplotlib.pyplot as plt

# Charger le fichier CSV
file_path = 'vehicles1.csv'
data = pd.read_csv(file_path)

# Filtrage des colonnes nécessaires
filtered_data = data[['vehicle_id', 'timestamp', 'latitude', 'longitude']]

# Suppression des doublons
filtered_data = filtered_data.drop_duplicates(subset=['vehicle_id', 'latitude', 'longitude', 'timestamp'])

# Gestion des valeurs manquantes
filtered_data = filtered_data.dropna()  # Supprimer les lignes avec des valeurs manquantes

# Conversion des timestamps en format datetime
filtered_data['timestamp'] = pd.to_datetime(filtered_data['timestamp'], errors='coerce')

# Suppression des lignes où la conversion du timestamp a échoué
filtered_data = filtered_data.dropna(subset=['timestamp'])

# Tri des données par vehicle_id et timestamp
filtered_data = filtered_data.sort_values(by=['vehicle_id', 'timestamp'])

# Sauvegarde des données prétraitées dans un nouveau fichier CSV
output_file_path = 'vehicles_processed.csv'
filtered_data.to_csv(output_file_path, index=False)

print(f"Fichier prétraité enregistré sous : {output_file_path}")

# Fonction pour créer des séquences
def create_sequences(df, sequence_length=10):
    sequences = []
    labels = []

    # Regrouper les données par véhicule
    for vehicle_id in df['vehicle_id'].unique():
        vehicle_data = df[df['vehicle_id'] == vehicle_id].sort_values('timestamp')
        
        # Créer des séquences de longueur 'sequence_length'
        for i in range(len(vehicle_data) - sequence_length):
            sequence = vehicle_data.iloc[i:i+sequence_length][['latitude', 'longitude']].values
            label = vehicle_data.iloc[i + sequence_length][['latitude', 'longitude']].values
            sequences.append(sequence)
            labels.append(label)
    
    return np.array(sequences), np.array(labels)

# Créer les séquences avec les données filtrées
X, y = create_sequences(filtered_data)

# Vérification des types de données dans X et y
print(f"X dtype avant conversion: {X.dtype}")
print(f"y dtype avant conversion: {y.dtype}")

# Conversion en float32 si nécessaire
X = X.astype('float32')
y = y.astype('float32')

# Vérification des valeurs NaN ou infinies
print(f"NaN values in X: {np.isnan(X).sum()}")
print(f"NaN values in y: {np.isnan(y).sum()}")

# Remplacer les NaN par des zéros (si nécessaire)
X = np.nan_to_num(X)
y = np.nan_to_num(y)

# Normalisation des données (latitude et longitude)
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = np.zeros_like(X)

# Appliquer la normalisation sur les 2 dimensions (latitude et longitude) pour toutes les séquences
for i in range(X.shape[2]):  # Latitude et Longitude (les 2 dimensions)
    X_scaled[:, :, i] = scaler.fit_transform(X[:, :, i])

# Vérification des données normalisées
print(f"Sample of X_scaled data after scaling: {X_scaled[:5]}")

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Vérification des types de données dans X_train et y_train après la normalisation
print(f"X_train dtype après normalisation: {X_train.dtype}")
print(f"y_train dtype après normalisation: {y_train.dtype}")

# Construction du modèle LSTM bidirectionnel
model = Sequential()

# LSTM bidirectionnel
model.add(Bidirectional(LSTM(64, return_sequences=False), input_shape=(X_train.shape[1], X_train.shape[2])))

# Couche de sortie pour prédire latitude et longitude (2 valeurs)
model.add(Dense(2))

# Compilation du modèle
model.compile(optimizer='adam', loss='mse')

# Entraînement du modèle
history = model.fit(X_train, y_train, epochs=5, batch_size=16, validation_data=(X_test, y_test))

# Prédiction sur les données de test
y_pred = model.predict(X_test)

# Inverse de la normalisation pour revenir aux valeurs réelles
# Reshape y_test et y_pred en 2D pour l'inverse de la normalisation
y_test_reshaped = y_test.reshape(-1, 2)  # Applatir y_test
y_pred_reshaped = y_pred.reshape(-1, 2)  # Applatir y_pred

# Inverser la normalisation
y_test_actual = scaler.inverse_transform(y_test_reshaped)
y_pred_actual = scaler.inverse_transform(y_pred_reshaped)

# Affichage des résultats (latitude vs longitude)
plt.figure(figsize=(12, 6))

# Latitude
plt.subplot(1, 2, 1)
plt.plot(y_test_actual[:, 0], label='True Latitude')
plt.plot(y_pred_actual[:, 0], label='Predicted Latitude', linestyle='--')
plt.title('Latitude Prediction')
plt.xlabel('Sample Index')
plt.ylabel('Latitude')
plt.legend()

# Longitude
plt.subplot(1, 2, 2)
plt.plot(y_test_actual[:, 1], label='True Longitude')
plt.plot(y_pred_actual[:, 1], label='Predicted Longitude', linestyle='--')
plt.title('Longitude Prediction')
plt.xlabel('Sample Index')
plt.ylabel('Longitude')
plt.legend()

plt.tight_layout()
plt.show()

# Affichage de la courbe de perte pour évaluer l'apprentissage
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


Fichier prétraité enregistré sous : vehicles_processed.csv
X dtype avant conversion: float64
y dtype avant conversion: object
NaN values in X: 0
NaN values in y: 0
Sample of X_scaled data after scaling: [[[0.8065262  0.83099365]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]]

 [[0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80493164 0.8371887 ]]

 [[0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80677795 0.8298645 ]
  [0.80493164 0.8371887 ]
  [0.81614685 0.8417053 ]]

 [[0.806777

C:\Users\yidir\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3052/3052 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - loss: 1075.8793 - val_loss: 0.0028
Epoch 2/5
3052/3052 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 0.0028 - val_loss: 0.0029
Epoch 3/5
3052/3052 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 0.0028 - val_loss: 0.0029
Epoch 4/5
3052/3052 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 0.0030 - val_loss: 0.0030
Epoch 5/5
3052/3052 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 0.0031 - val_loss: 0.0030
382/382 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


ValueError: operands could not be broadcast together with shapes (12205,2) (10,) (12205,2) 